In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import os

import numpy as np
import pandas as pd
import tensorflow as tf

import atecml.data

from contextlib import contextmanager
from tqdm import tqdm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.externals import joblib

plt.style.use('ggplot')

In [2]:
#Loading Data....
data = atecml.data.load_train()
predictors = [x for x in data.columns if x not in atecml.data.NOT_FEATURE_SUM]


feature_tree_num = 1
train_model =[]
for idx in range(0,1):
    for item in ['dart','gbdt','rf']:
        for feature_grp_idx in range(0,feature_tree_num):
            for target in ['Normal','Fraud']:
                train_id = item + '__'+target +'__'+str(idx) +'__' + str(feature_grp_idx)
                train_model.append(train_id)

                
def model_load(model_name):
    model_cache_name = './'+model_name+'.model'
    if (os.path.exists(model_cache_name)):
        clf = joblib.load(model_cache_name)
    else:
        print('ERROR...............')
    return clf


if (os.path.exists('./feature_list.dat')):
    print('Load Feature List from persistant store...')
    feature_list = joblib.load('./feature_list.dat')
else:
    print('Error: Feature Cache File missing...')


trained_model_list =[]
with atecml.data.timer('Classification: Model Training'):
    for train_id in tqdm(range(len(train_model))):
        fit_model = model_load(train_model[train_id])
        trained_model_list.append(fit_model)


100%|██████████| 6/6 [00:00<00:00, 41.90it/s]

Load Feature List from persistant store...
[2018-06-27 16:52:30][Classification: Model Training] Begin ...
[2018-06-27 16:52:30][Classification: Model Training] End   ...[Elapsed: 0.15s]


In [3]:
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
extend_array=pd.DataFrame()
with atecml.data.timer('Feature Extraction'):
    for idx in tqdm(range(len(trained_model_list))):
        clf = trained_model_list[idx]
        model_params = train_model[idx].split('__')
        f_idx = int(model_params[3])
        select_feature = feature_list[f_idx]
        enc = OneHotEncoder()
        print(train_model[idx],"   > Fitting Encoder...")
        enc.fit(clf.apply(data[select_feature]))
        print(train_model[idx],"   > Encoder Transform...")
        ext_feature =enc.transform(clf.apply(data[select_feature]))
        print(train_model[idx],"   > Load Extension Features to DataFrame...")
        ext_feature_df = pd.DataFrame(ext_feature.toarray()).T
        extend_array.append(ext_feature_df)


'\nwith atecml.data.timer(\'Feature Extraction\'):\n    for idx in tqdm(range(len(trained_model_list))):\n        clf = trained_model_list[idx]\n        model_params = train_model[idx].split(\'__\')\n        f_idx = int(model_params[3])\n        select_feature = feature_list[f_idx]\n        enc = OneHotEncoder()\n        print(train_model[idx],"   > Fitting Encoder...")\n        enc.fit(clf.apply(data[select_feature]))\n        print(train_model[idx],"   > Encoder Transform...")\n        ext_feature =enc.transform(clf.apply(data[select_feature]))\n        print(train_model[idx],"   > Load Extension Features to DataFrame...")\n        ext_feature_df = pd.DataFrame(ext_feature.toarray()).T\n        extend_array.append(ext_feature_df)\n'

In [4]:
extend_array=pd.DataFrame()

In [5]:
idx = 1

In [10]:
clf = trained_model_list[idx]
model_params = train_model[idx].split('__')
f_idx = int(model_params[3])
select_feature = feature_list[f_idx]

enc = OneHotEncoder()
print(train_model[idx],"   > Fitting Encoder...")
enc.fit(clf.apply(data[select_feature]))
print(train_model[idx],"   > Encoder Transform...")
ext_feature =enc.transform(clf.apply(data[select_feature]))


dart__Fraud__0__0    > Fitting Encoder...
dart__Fraud__0__0    > Encoder Transform...


In [11]:
print(train_model[idx],"   > Load OneHotEncoder Result to np.array...")
ext_feature_arr =ext_feature.toarray()

dart__Fraud__0__0    > Load OneHotEncoder Result to np.array...


In [12]:
print(train_model[idx],"   > Transform to DataFrame...")
ext_feature_df = pd.DataFrame(ext_feature_arr)

dart__Fraud__0__0    > Transform to DataFrame...


In [20]:
extend_array =pd.concat([extend_array,ext_feature_df.T],axis=0)

In [17]:
extend_array =pd.concat([extend_array,ext_feature_df.T],axis=0)

Series([], dtype: object)

In [22]:
np.stack([ext_feature_arr,ext_feature_arr], axis=1).shape

(994731, 2, 6400)

In [ ]:

ext_feature_df = pd.DataFrame().T